In [66]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, r2_score, accuracy_score
from sklearn.svm import  SVC
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, ElasticNet
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import warnings
warnings.simplefilter('ignore')
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, BaggingRegressor, RandomForestClassifier, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor, GradientBoostingClassifier, StackingClassifier, RandomForestRegressor, StackingRegressor
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor, LGBMClassifier

In [67]:
train = pd.read_csv("./BlackFriday/train.csv")
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [68]:
test = pd.read_csv("./BlackFriday/test_Vges7qu.csv")
test.head()

,Comb,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004P00128942,1000004,P00128942,M,46-50,7,B,2,0,1,11.0,NaN
1,1000009P00113442,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010P00288442,1000010,P00288442,F,36-45,1,B,4+,0,5,14.0,NaN
3,1000010P00145342,1000010,P00145342,F,36-45,1,B,4+,0,4,9.0,NaN
4,1000011P00053842,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [69]:
train = train.drop(['User_ID','Product_ID'], axis=1)
X = train.drop('Purchase',axis=1)
y = train['Purchase']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=24)

In [70]:
X_test = test.drop(['User_ID','Product_ID','Comb'], axis=1)

In [71]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [72]:
X_train['Product_Category_2'] = X_train['Product_Category_2'].astype(int)
X_train['Product_Category_3'] = X_train['Product_Category_3'].astype(int)

In [73]:
X_train = X_train.astype(str)

In [74]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')
X_trn_ohe = ohe.fit_transform(X_train)


In [75]:
X_test['Product_Category_2'] = X_test['Product_Category_2'].astype(int)
X_test['Product_Category_3'] = X_test['Product_Category_3'].astype(int)

In [76]:
X_test = X_test.astype(str)

In [77]:
# ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')
X_tst_ohe = ohe.transform(X_test)

In [78]:
X_tst_ohe.shape, X_trn_ohe.shape

((233599, 94), (385047, 94))

# DecisionTreeRegressor()

In [79]:
dtr = DecisionTreeRegressor(random_state=24, max_depth=8)
dtr.fit(X_trn_ohe, y_train)
y_pred = dtr.predict(X_tst_ohe)
y_pred

array([13147.17511661, 10086.95272727,  6209.91902926, ...,
       13147.17511661, 19825.21340131,  6209.91902926])

In [80]:
submission = pd.read_csv("./BlackFriday/sample_submission_V9Inaty.csv")
submission.head()

,Purchase,User_ID,Product_ID
0,100,1000004,P00128942
1,100,1000009,P00113442
2,100,1000010,P00288442
3,100,1000010,P00145342
4,100,1000011,P00053842


In [81]:
submission['Purchase'] = y_pred

In [82]:
submission.head()


,Purchase,User_ID,Product_ID
0,13147.175117,1000004,P00128942
1,10086.952727,1000009,P00113442
2,6209.919029,1000010,P00288442
3,6209.919029,1000010,P00145342
4,6209.919029,1000011,P00053842


In [83]:
submission.to_csv("./submit.csv", index=False)

In [84]:
from catboost import CatBoostRegressor

In [85]:

gbm = CatBoostRegressor(random_state=24, n_estimators = 100, learning_rate=0.1, max_depth=8)
gbm.fit(X_trn_ohe,y_train)
y_pred = gbm.predict(X_tst_ohe)
y_pred

0:	learn: 4744.8522631	total: 47.8ms	remaining: 4.73s
1:	learn: 4507.7474721	total: 90.8ms	remaining: 4.45s
2:	learn: 4305.5880073	total: 124ms	remaining: 4.01s
3:	learn: 4133.7405861	total: 161ms	remaining: 3.86s
4:	learn: 3988.9153934	total: 196ms	remaining: 3.72s
5:	learn: 3860.2092402	total: 232ms	remaining: 3.63s
6:	learn: 3753.0716441	total: 281ms	remaining: 3.73s
7:	learn: 3657.7533993	total: 316ms	remaining: 3.63s
8:	learn: 3579.8235787	total: 348ms	remaining: 3.52s
9:	learn: 3513.6429982	total: 384ms	remaining: 3.45s
10:	learn: 3456.2878230	total: 418ms	remaining: 3.38s
11:	learn: 3407.7680227	total: 458ms	remaining: 3.36s
12:	learn: 3365.0421627	total: 500ms	remaining: 3.35s
13:	learn: 3329.4842239	total: 545ms	remaining: 3.35s
14:	learn: 3298.3457136	total: 585ms	remaining: 3.31s
15:	learn: 3262.5182357	total: 631ms	remaining: 3.31s
16:	learn: 3232.7107294	total: 671ms	remaining: 3.28s
17:	learn: 3211.3054084	total: 710ms	remaining: 3.23s
18:	learn: 3193.8375595	total: 746ms

array([13446.68519688, 10273.95249667,  6166.42248328, ...,
       12946.36660404, 19749.44918497,  2149.78679309])

In [86]:
submission['Purchase'] = y_pred
submission.head()

,Purchase,User_ID,Product_ID
0,13446.685197,1000004,P00128942
1,10273.952497,1000009,P00113442
2,6166.422483,1000010,P00288442
3,2496.558226,1000010,P00145342
4,2730.201521,1000011,P00053842


In [87]:

submission.to_csv("./submit2.csv", index=False)


In [88]:
from lightgbm import LGBMRegressor


# LGBMRegressor()

In [89]:
gbm = LGBMRegressor(random_state=24, n_estimators = 100, learning_rate=0.1, max_depth=8)
gbm.fit(X_trn_ohe,y_train)
y_pred = gbm.predict(X_tst_ohe) 
y_pred

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011226 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 385047, number of used features: 94
[LightGBM] [Info] Start training from score 9261.376115


array([13637.07983153, 10626.07170927,  6078.88282596, ...,
       13080.24633918, 19973.91952859,  2065.57667458])

In [90]:
submission['Purchase'] = y_pred
submission.head()
submission.to_csv("./submit2.csv", index=False)


# XGBRegressor()

In [91]:
from xgboost import XGBRegressor


In [92]:
gbm = XGBRegressor(random_state=24, n_estimators = 100, learning_rate=0.1, max_depth=5)
gbm.fit(X_trn_ohe,y_train)
y_pred = gbm.predict(X_tst_ohe)

In [93]:
y_pred

array([13226.5205, 10407.616 ,  6138.631 , ..., 13208.643 , 19704.055 ,
        2320.499 ], dtype=float32)

In [94]:
submission['Purchase'] = y_pred
submission.head()
submission.to_csv("./submit3.csv", index=False)

----------------------------------

# StackRegressor()

In [95]:


rf = RandomForestRegressor()
dtc = DecisionTreeRegressor()
lgbm = LGBMRegressor()

gbm = CatBoostRegressor()

stack = StackingRegressor(estimators=[('RF', rf),('DTC',dtc),('LGBM',lgbm)], final_estimator=gbm, passthrough=True)
stack.fit(X_trn_ohe, y_train)
y_pred = stack.predict(X_tst_ohe)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010204 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 385047, number of used features: 94
[LightGBM] [Info] Start training from score 9261.376115
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009848 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 308037, number of used features: 94
[LightGBM] [Info] Start training from score 9259.068407
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009789 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

In [96]:
submission['Purchase'] = y_pred
submission.head()
submission.to_csv("./submit4.csv", index=False)

In [99]:

rf = RandomForestRegressor(random_state=24, n_estimators = 50)
dtc = DecisionTreeRegressor(random_state=24)
lgbm = LGBMRegressor(random_state=24, n_estimators = 50)

gbm = CatBoostRegressor(random_state=24, n_estimators = 50)

stack = StackingRegressor(estimators=[('RF', rf),('DTC',dtc),('LGBM',lgbm)], final_estimator=gbm, passthrough=True)
stack.fit(X_trn_ohe, y_train)
y_pred = stack.predict(X_tst_ohe)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011022 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 385047, number of used features: 94
[LightGBM] [Info] Start training from score 9261.376115
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006139 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 308037, number of used features: 94
[LightGBM] [Info] Start training from score 9259.068407
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008870 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

In [100]:
submission['Purchase'] = y_pred
submission.head()
submission.to_csv("./submit5.csv", index=False)